# LangChain Expression Language (LCEL)

In [1]:
import openai
import os

In [2]:
openai.api_key = open("openAI_key.txt", "r").read().strip("\n")

In [18]:
#!pip install pydantic==1.10.9
#!pip install langchain==0.0.336
!pip install docarray==0.39.1  

  Using cached docarray-0.39.1-py3-none-any.whl.metadata (36 kB)
Using cached docarray-0.39.1-py3-none-any.whl (265 kB)
  Attempting uninstall: docarray
    Found existing installation: docarray 0.16.5
    Uninstalling docarray-0.16.5:
      Successfully uninstalled docarray-0.16.5


In [4]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

## Simple Chain

In [5]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
model = ChatOpenAI(openai_api_key=openai.api_key)
output_parser = StrOutputParser()

In [6]:
chain = prompt | model | output_parser

In [7]:
chain.invoke({"topic": "bears"})

"Why don't bears wear shoes?\n\nBecause they have bear feet!"

## More complex chain

And Runnable Map to supply user-provided inputs to the prompt.

In [8]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [19]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(openai_api_key=openai.api_key)
)
retriever = vectorstore.as_retriever()

ImportError: Could not import docarray python package. Please install it with `pip install "langchain[docarray]"`.

In [ ]:
retriever.get_relevant_documents("where did harrison work?")

In [ ]:
retriever.get_relevant_documents("what do bears like to eat")

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain.schema.runnable import RunnableMap

In [ ]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [ ]:
chain.invoke({"question": "where did harrison work?"})

In [ ]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [ ]:
inputs.invoke({"question": "where did harrison work?"})

## Bind

and OpenAI Functions

In [20]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [22]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model = ChatOpenAI(temperature=0, openai_api_key=openai.api_key).bind(functions=functions)

In [23]:
runnable = prompt | model

In [24]:
runnable.invoke({"input": "what is the weather in sf"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'weather_search', 'arguments': '{\n  "airport_code": "SFO"\n}'}})

In [25]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

In [26]:
model = model.bind(functions=functions)

In [27]:
runnable = prompt | model

In [28]:
runnable.invoke({"input": "how did the patriots do yesterday?"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'sports_search', 'arguments': '{\n  "team_name": "patriots"\n}'}})

## Fallbacks

In [29]:
from langchain.llms import OpenAI
import json

In [31]:
simple_model = OpenAI(
    temperature=0, 
    max_tokens=1000, 
    model="text-davinci-001", 
    openai_api_key=openai.api_key
)
simple_chain = simple_model | json.loads

In [32]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

In [33]:
simple_model.invoke(challenge)

'\n\n["The Waste Land","T.S. Eliot","April is the cruelest month, breeding lilacs out of the dead land"]\n\n["The Raven","Edgar Allan Poe","Once upon a midnight dreary, while I pondered, weak and weary"]\n\n["Ode to a Nightingale","John Keats","Thou still unravish\'d bride of quietness, Thou foster-child of silence and slow time"]'

Note: The next line is expected to fail.

In [35]:
simple_chain.invoke(challenge)

JSONDecodeError: Extra data: line 5 column 1 (char 103)

In [36]:
model = ChatOpenAI(temperature=0, openai_api_key=openai.api_key)
chain = model | StrOutputParser() | json.loads

In [37]:
chain.invoke(challenge)

{'poem1': {'title': 'Whispers of the Wind',
  'author': 'Emily Rivers',
  'first_line': 'Softly it comes, the whisper of the wind'},
 'poem2': {'title': 'Silent Serenade',
  'author': 'Jacob Moore',
  'first_line': 'In the stillness of night, a silent serenade'},
 'poem3': {'title': 'Dancing Shadows',
  'author': 'Sophia Anderson',
  'first_line': 'Shadows dance upon the moonlit floor'}}

In [38]:
final_chain = simple_chain.with_fallbacks([chain])

In [39]:
final_chain.invoke(challenge)

{'poem1': {'title': 'Whispers of the Wind',
  'author': 'Emily Rivers',
  'first_line': 'Softly it comes, the whisper of the wind'},
 'poem2': {'title': 'Silent Serenade',
  'author': 'Jacob Moore',
  'first_line': 'In the stillness of night, a silent serenade'},
 'poem3': {'title': 'Dancing Shadows',
  'author': 'Sophia Anderson',
  'first_line': 'Shadows dance upon the moonlit floor'}}

## Interface

In [41]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI(openai_api_key=openai.api_key)
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [42]:
chain.invoke({"topic": "bears"})

"Why don't bears wear shoes? \n\nBecause they have bear feet!"

In [43]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

["Why don't bears wear shoes?\n\nBecause they have bear feet!",
 'Why did the frog take the bus to work?\n\nBecause his car got toad away!']

In [44]:
for t in chain.stream({"topic": "bears"}):
    print(t)


Why
 don
't
 bears
 wear
 shoes
?
 


Because
 they
 have
 bear
 feet
!



In [45]:
response = await chain.ainvoke({"topic": "bears"})
response

"Why don't bears wear shoes? \n\nBecause they have bear feet!"

In [46]:
response = await chain.ainvoke({"topic": "bunnies"})
response

"Why don't bunnies make good storytellers? \n\nBecause they only have one tail to tell!"